In [14]:
import numpy as np
import pandas as pd
import math
import os
import sys
import yaml
from geopy.distance import great_circle
from geopy import Point

In [2]:
proj_dir = os.path.abspath('')
sys.path.append(proj_dir)
config_fp = os.path.join(proj_dir, 'config.yaml')

with open(config_fp, 'r') as f:
    config = yaml.safe_load(f)

# ------------- Config parameters start ------------- #
data_dir = config['filepath']['data_dir']
model_dir = config['filepath']['model_dir']
bihar_pkl_fp = data_dir + config['filepath']['bihar_pkl_fp']
bihar_npy_fp = data_dir + config['filepath']['bihar_npy_fp']
bihar_locations_fp = data_dir + config['filepath']['bihar_locations_fp']
china_npy_fp = data_dir + config['filepath']['china_npy_fp']
china_locations_fp = data_dir + config['filepath']['china_locations_fp']
bihar_map_fp = data_dir + config['filepath']['bihar_map_fp']

batch_size = int(config['train']['batch_size'])
num_epochs = int(config['train']['num_epochs'])
forecast_window = int(config['train']['forecast_window'])
hist_window = int(config['train']['hist_window'])
hidden_dim = int(config['train']['hidden_dim'])
lr = float(config['train']['lr'])
model_type = config['train']['model']

update = int(config['dataset']['update'])
data_start = config['dataset']['data_start']
data_end = config['dataset']['data_end']

train_start = config['split']['train_start']
train_end = config['split']['train_end']
val_start = config['split']['val_start']
val_end = config['split']['val_end']
test_start = config['split']['test_start']
test_end = config['split']['test_end']
# ------------- Config parameters end   ------------- #


In [3]:
def process_locs(loc_fp):
    locs = []

    with open(loc_fp, 'r') as f:
        
        for line in f:
            data = line.strip().split('|')
            locs.append([float(data[-2]), float(data[-1])])

    return locs

bihar_locs = process_locs(bihar_locations_fp)

In [5]:
# bihar_locs

In [45]:
def angleFromCoordinate(loc_1, loc_2):
    # loc: [longitude, latitude]
    dLon = (loc_2[0] - loc_1[0])

    y = math.sin(dLon) * math.cos(loc_2[1])
    x = math.cos(loc_1[1]) * math.sin(loc_2[1]) - math.sin(loc_1[1]) * math.cos(loc_2[1]) * math.cos(dLon)

    brng = math.atan2(y, x)

    brng = math.degrees(brng)
    brng = (brng + 360) % 360
    # brng = 360 - brng                       # count degrees clockwise - remove to make counter-clockwise

    return brng

In [46]:
angles = np.zeros((len(bihar_locs), len(bihar_locs)))

for i, loc_x in enumerate(bihar_locs):
    for j, loc_y in enumerate(bihar_locs):
        if i == j: continue

        angles[i, j] = angleFromCoordinate(loc_x, loc_y)

# Wind Magnitude along edge

$v . cos(\alpha - 90) + u . cos(\alpha)$

In [24]:
bihar_locs = np.array(bihar_locs)

In [34]:
imputed_df = pd.read_pickle(f'{data_dir}/bihar_meteo_era5_may_jan_iterative_imputed.pkl')
cols = list(imputed_df.columns)


loc_0_df = imputed_df[(imputed_df['longitude'] == bihar_locs[0][0]) & (imputed_df['latitude'] == bihar_locs[0][1])]
loc_1_df = imputed_df[(imputed_df['longitude'] == bihar_locs[1][0]) & (imputed_df['latitude'] == bihar_locs[1][1])]
# loc_0_df, loc_1_df

['timestamp',
 'block',
 'district',
 'latitude',
 'longitude',
 'rh',
 'temp',
 'blh',
 'u10',
 'v10',
 'kx',
 'sp',
 'tp',
 'pm25']

In [42]:
u10_col, v10_col = cols.index('u10'), cols.index('v10')

loc_0_u = loc_0_df.to_numpy()[0, u10_col]
loc_0_v = loc_0_df.to_numpy()[0, v10_col]

loc_1_u = loc_1_df.to_numpy()[0, u10_col]
loc_1_v = loc_1_df.to_numpy()[0, v10_col]

In [47]:
alpha = np.pi * angles[0][1] / 180.0
loc_0_v * math.cos(alpha - np.pi/2) + loc_0_u * math.cos(alpha)

0.8138410367304115

In [48]:
alpha = np.pi * angles[1][0] / 180.0
loc_1_v * math.cos(alpha - np.pi/2) + loc_1_u * math.cos(alpha)

-0.8056188535068237